# Notebook 2: Modeling of full dataset

## Overview

This notebook takes in the reshaped data, with lemmatized sentences from the prior 'Data Cleaning and NLP Preprocessing' notebook & fully models the dataset, returning a dataframe of all modeling inputs, hyperparameters and results.

Steps performed by the included functions:


- import dataset that has been preprocessed with Spacy and reshaped to have each document-sentence represented only once 
- train-test-split the preprocessed dataset & return a dictionary of processed and split data that is ready for modeling
- create stopword lists
- run modeling via logistic or random forest regression methods (defaults are set for logistic regression) - modeling is via gridsearch, and a dataframe is returned, including all modeling inputs, hyperparameters and results.
- create a coefficient/beta bar chart for each Cash Trap Trigger category
- create a confusion matrix for each Cash Trap Trigger category

## Import packages

In [17]:
import pandas as pd
import numpy as np
np.random.seed(99)
RANDOM_STATE = 99

from spacy.lang.en.stop_words import STOP_WORDS

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

import time
import pickle

## Defining Modeling Functions

In [2]:
def get_reshaped_lemmatized():
    # Import the CSV file
    df = pd.read_csv('../data/reshaped_lemmatized.csv')
    return df

In [3]:
# function to split data for each target column (trigger type)
def get_split_data(df, target_info):
    # extract target name
    target = target_info['target']
    model_input  = target_info['model_input']
    
    # create X, Y
    X = df[model_input]
    y = df[target]
    indices = df.index

    print(f"Number of distinct labeled {target} document/sentence combinations within the full data set: {y.value_counts()[1]}")   
    
    y = y.astype('int')

    # run test, train split
    X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size = 0.3, stratify = y, random_state = RANDOM_STATE)

    
    # create output dictionary
    split_data = {}
    split_data['X_train'] = X_train
    split_data['X_test'] = X_test
    split_data['y_train'] = y_train
    split_data['y_test'] = y_test
    split_data['indices_train'] = indices_train
    split_data['indices_test'] = indices_test
    
    # Return the train-test-split data in a dictionary form
    return split_data

In [4]:
# Incorporate Stopwords
def get_stopwords():
    # might need space
    short_stopwords = ['the', 'to', 'of', 'be', 'and', 'in', 'a', 'marriott']
    short_stopwords2 = ['the', 'and', 'a', 'to', 'it', 'be', 'for', 'with', 'that', 'marriott']
    stopwords = list(STOP_WORDS) + ['marriott']

    return short_stopwords, short_stopwords2, stopwords

In [5]:
# run the model with the chosen model and metric
def run_model(split_data, target_info):

    # get stopwords
    short_stopwords, short_stopwords2, stopwords = get_stopwords()
    
    # set pipeline according to the selected model:
    if target_info['model'] == 'lr':
        # Define CVEC + Logistic Regression Pipeline
        pipe = Pipeline([('cvec', CountVectorizer()), ('lr', LogisticRegression(solver = 'liblinear', random_state = RANDOM_STATE))])
        params = {
            'cvec__ngram_range': [(1,2), (1,3), (1,4), (1,5), (1,6), (1,7)],
            'cvec__stop_words': [short_stopwords, short_stopwords2, stopwords],  
            'cvec__max_features': [100, 200, 400, 600, 1000],
            'cvec__min_df': [2],
            'cvec__max_df': [.99],
            }

    elif target_info['model'] == 'rf':
        # Define CVEC + Logistic Regression Pipeline
        pipe = Pipeline([('cvec', CountVectorizer()), ('rf', RandomForestClassifier(random_state = RANDOM_STATE, n_jobs = 2))])
        params = {
            'cvec__ngram_range': [(1,2), (1,3), (1,4), (1,5)],
            'cvec__stop_words': [short_stopwords, short_stopwords2, stopwords],  
            'cvec__max_features': [100, 200, 400, 800],
            'cvec__min_df': [2],
            'cvec__max_df': [.99],
            'rf__max_depth': [4,5, 6],
            'rf__min_samples_split': [2,3],
            'rf__min_samples_leaf': [10, 12]
            }

    
    else:
        print('Error: did not specify model')

    # define pipeline
    gs_model = GridSearchCV(pipe, param_grid = params, cv = 3, scoring = target_info['metric'])

    # Start the timer.
    t0 = time.time()

    # extract X_train and y_train
    X_train = split_data['X_train']
    y_train = split_data['y_train']
    
    # run pipeline
    model_result = gs_model.fit(X_train, y_train)

    print(f"Seconds elapsed for fitting: {(time.time() - t0):.3f}") # How many seconds elapsed.   
    return model_result

## Perform all modeling steps for all targets

Note: in advance, adjust parameters/metrics for any particular modeling target in the target_dict below

In [6]:
# Control of the main project

# define dictionary of targets contains: tag, model, metric, input, order
target_dict = {}
target_dict['nontrigger'] = {'target': 'nontrigger', 'model': 'lr', 'metric': 'roc_auc', 'model_input': 'SentenceLemmas'}
target_dict['loan_default'] = {'target': 'loan_default', 'model': 'lr', 'metric':'roc_auc', 'model_input':'SentenceLemmas'}
target_dict['unspecified'] = {'target': 'unspecified', 'model': 'lr', 'metric':'roc_auc', 'model_input':'SentenceLemmas'}
target_dict['debt_yield_fall'] = {'target': 'debt_yield_fall', 'model': 'lr', 'metric': 'roc_auc', 'model_input': 'SentenceLemmas'}
target_dict['mezzanine_default'] = {'target': 'mezzanine_default', 'model': 'lr', 'metric': 'roc_auc', 'model_input': 'SentenceLemmas'}
target_dict['bankruptcy'] = {'target': 'bankruptcy', 'model': 'lr', 'metric': 'roc_auc', 'model_input': 'SentenceLemmas'}
target_dict['tenant_failure'] = {'target': 'tenant_failure', 'model': 'lr', 'metric': 'roc_auc', 'model_input': 'SentenceLemmas'}
target_dict['renovations'] = {'target': 'renovations', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
target_dict['aggregate_debt_yield_fall'] = {'target': 'aggregate_debt_yield_fall', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
target_dict['dscr_fall'] = {'target': 'dscr_fall', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
target_dict['operator_termination'] = {'target': 'operator_termination', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
target_dict['sponsor_termination'] = {'target': 'sponsor_termination', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
target_dict['sff'] = {'target': 'sff', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
target_dict['mezzanine_outstanding'] = {'target': 'mezzanine_outstanding', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
target_dict['aggregate_dscr_fall'] = {'target': 'aggregate_dscr_fall', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
target_dict['delayed_repayment'] = {'target': 'delayed_repayment', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}


# set output_dict - will contain target + output of calculations
output_dict = {}

# get data
df = get_reshaped_lemmatized()

# run for each model definition
for k,v in target_dict.items():
    
    print(f"{target_dict[k]['target'].replace('_', ' ').title()}: creating Train-Test split")
    # get split data
    split_data = get_split_data(df, v)
    
    print(f"Model fit in progress: {target_dict[k]}")
    # run model
    model_result = run_model(split_data, v)
    
    # make the output dictionary
    output_dict[k] = v
    output_dict[k]['split_data'] = split_data
    output_dict[k]['model_result'] = model_result

    
    print(f"Best fit parameters: {model_result.best_params_}")
    print(f"Best fit 3-fold cross validation score: {model_result.best_score_:.3f}")
    print(f"{target_dict[k]['target'].replace('_', ' ').title()} {target_dict[k]['metric']} Train score: {model_result.score(split_data['X_train'], split_data['y_train']):.3f}")
    print(f"{target_dict[k]['target'].replace('_', ' ').title()} {target_dict[k]['metric']} Test score: {model_result.score(split_data['X_test'], split_data['y_test']):.3f}")
    print("\n")
    
    output_dict[k]['best_params'] = model_result.best_params_
    output_dict[k]['count_deduplicated'] = df[target_dict[k]['target']].value_counts()[1]
    output_dict[k]['test_data_count'] = output_dict[k]['split_data']['y_test'].value_counts()[1]
    output_dict[k]['best_crossval_score'] = model_result.best_score_
    output_dict[k]['train_score'] = model_result.score(split_data['X_train'], split_data['y_train'])
    output_dict[k]['test_score'] = model_result.score(split_data['X_test'], split_data['y_test'])                                                  
    
    '''
    outfile = open(f"../data/models/{target_dict[k]['target']}_pickle", 'wb')
    pickle.dump(model_result, outfile)
    outfile.close()
    '''
    

full_output_dict = [output_dict[key] for key in output_dict.keys()]
results_df = pd.DataFrame.from_dict(full_output_dict)

# Export summary table
results_df.to_csv(f'../data/exported_data/results_df_fulldata.csv')

print("Modeling complete!")


Nontrigger: creating Train-Test split
Number of distinct labeled nontrigger document/sentence combinations within the full data set: 1030
Model fit in progress: {'target': 'nontrigger', 'model': 'lr', 'metric': 'roc_auc', 'model_input': 'SentenceLemmas'}
Seconds elapsed for fitting: 144.864
Best fit parameters: {'cvec__max_df': 0.99, 'cvec__max_features': 1000, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 3), 'cvec__stop_words': ['something', 'whereafter', 'are', 'full', 'of', 'neither', 'since', 'sixty', 'twelve', 'here', 'it', 'she', 'a', 'do', 'everyone', 'this', 'bottom', 'used', 'anywhere', 'never', 'the', 'mine', 'thru', 'either', 'give', 'serious', 'more', 'my', 'seems', 'just', 'i', 'formerly', 'namely', 'through', 'why', 'empty', 'yet', 'mostly', 'if', 'they', 'will', 'sometime', 'without', 'amount', 'get', 'has', 'nine', 'quite', 'does', 'though', 'whereupon', 'very', 'sometimes', 'whose', 'eleven', 'otherwise', 'side', 'made', 'me', 'hence', 'however', 'next', 'nor', 'whateve

Debt Yield Fall roc_auc Test score: 0.997


Mezzanine Default: creating Train-Test split
Number of distinct labeled mezzanine_default document/sentence combinations within the full data set: 72
Model fit in progress: {'target': 'mezzanine_default', 'model': 'lr', 'metric': 'roc_auc', 'model_input': 'SentenceLemmas'}
Seconds elapsed for fitting: 145.851
Best fit parameters: {'cvec__max_df': 0.99, 'cvec__max_features': 100, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 2), 'cvec__stop_words': ['something', 'whereafter', 'are', 'full', 'of', 'neither', 'since', 'sixty', 'twelve', 'here', 'it', 'she', 'a', 'do', 'everyone', 'this', 'bottom', 'used', 'anywhere', 'never', 'the', 'mine', 'thru', 'either', 'give', 'serious', 'more', 'my', 'seems', 'just', 'i', 'formerly', 'namely', 'through', 'why', 'empty', 'yet', 'mostly', 'if', 'they', 'will', 'sometime', 'without', 'amount', 'get', 'has', 'nine', 'quite', 'does', 'though', 'whereupon', 'very', 'sometimes', 'whose', 'eleven', 'otherwise', 'si

Aggregate Debt Yield Fall: creating Train-Test split
Number of distinct labeled aggregate_debt_yield_fall document/sentence combinations within the full data set: 19
Model fit in progress: {'target': 'aggregate_debt_yield_fall', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
Seconds elapsed for fitting: 147.594
Best fit parameters: {'cvec__max_df': 0.99, 'cvec__max_features': 600, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 3), 'cvec__stop_words': ['the', 'and', 'a', 'to', 'it', 'be', 'for', 'with', 'that', 'marriott']}
Best fit 3-fold cross validation score: 0.546
Aggregate Debt Yield Fall f1 Train score: 1.000
Aggregate Debt Yield Fall f1 Test score: 0.800


Dscr Fall: creating Train-Test split
Number of distinct labeled dscr_fall document/sentence combinations within the full data set: 23
Model fit in progress: {'target': 'dscr_fall', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
Seconds elapsed for fitting: 149.085
Best fit parameters: {'cvec__max

Sff: creating Train-Test split
Number of distinct labeled sff document/sentence combinations within the full data set: 9
Model fit in progress: {'target': 'sff', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
Seconds elapsed for fitting: 147.548
Best fit parameters: {'cvec__max_df': 0.99, 'cvec__max_features': 100, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 2), 'cvec__stop_words': ['the', 'and', 'a', 'to', 'it', 'be', 'for', 'with', 'that', 'marriott']}
Best fit 3-fold cross validation score: 0.222
Sff f1 Train score: 0.800
Sff f1 Test score: 0.000


Mezzanine Outstanding: creating Train-Test split
Number of distinct labeled mezzanine_outstanding document/sentence combinations within the full data set: 8
Model fit in progress: {'target': 'mezzanine_outstanding', 'model': 'lr', 'metric': 'f1', 'model_input': 'SentenceLemmas'}
Seconds elapsed for fitting: 143.265
Best fit parameters: {'cvec__max_df': 0.99, 'cvec__max_features': 400, 'cvec__min_df': 2, 'cvec__ngram_range

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


Seconds elapsed for fitting: 141.386
Best fit parameters: {'cvec__max_df': 0.99, 'cvec__max_features': 100, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 2), 'cvec__stop_words': ['the', 'to', 'of', 'be', 'and', 'in', 'a', 'marriott']}
Best fit 3-fold cross validation score: 0.000
Delayed Repayment f1 Train score: 1.000
Delayed Repayment f1 Test score: 0.000


Modeling complete!


In [7]:
# Show results of modeling (already exported as 'results_df_fulldata.csv')
results_df


,target,model,metric,model_input,split_data,model_result,best_params,count_deduplicated,test_data_count,best_crossval_score,train_score,test_score
0,nontrigger,lr,roc_auc,SentenceLemmas,{'X_train': [' trigger period mean a period a ...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 1...",1030,309,0.986957,0.999823,0.985937
1,loan_default,lr,roc_auc,SentenceLemmas,{'X_train': ['upon the occurrence of a lockbox...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 4...",553,166,0.982374,0.999221,0.991220
2,unspecified,lr,roc_auc,SentenceLemmas,{'X_train': ['any fund remain in the reserve a...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 1...",498,149,0.965063,0.999828,0.964248
3,debt_yield_fall,lr,roc_auc,SentenceLemmas,{'X_train': [' approve operating expense mean ...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 2...",188,56,0.974742,0.999959,0.997085
4,mezzanine_default,lr,roc_auc,SentenceLemmas,{'X_train': ['borrower hereby represent and wa...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 1...",72,22,0.949262,1.000000,0.994850
5,bankruptcy,lr,roc_auc,SentenceLemmas,{'X_train': ['hard lockbox and spring cash man...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 2...",44,13,0.832279,0.999897,0.995529
6,tenant_failure,lr,roc_auc,SentenceLemmas,{'X_train': ['follow the occurrence and prior ...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 2...",74,22,0.957538,1.000000,0.999863
7,renovations,lr,f1,SentenceLemmas,{'X_train': ['see description of the mortgage ...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 1...",26,8,0.755556,0.888889,0.933333
8,aggregate_debt_yield_fall,lr,f1,SentenceLemmas,{'X_train': ['the mortgage lender will make di...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 6...",19,6,0.546032,1.000000,0.800000
9,dscr_fall,lr,f1,SentenceLemmas,{'X_train': ['upon the write request of mortga...,"GridSearchCV(cv=3, error_score=nan,\n ...","{'cvec__max_df': 0.99, 'cvec__max_features': 1...",23,7,0.616667,1.000000,0.750000


In [19]:
# Pickle the model details
path = '../data/exported_data/'
outfile = open(path + 'results_df_notebook2.pkl', 'wb')
pickle.dump(results_df, outfile)
outfile.close()

outfile = open(path + 'full_output_dict_notebook2.pkl', 'wb')
pickle.dump(full_output_dict, outfile)
outfile.close()

